## Data Mining and Machine Learning
### Multiclass Logistic Regression
### Dataset:Vehicle
### Libraries: H20  and scikit-learn 
#### Edgar Acuna
#### November 2021
#### Datasets: Vehicle, Landsat

In [1]:
import h2o
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
from h2o.estimators.glm   import H2OGeneralizedLinearEstimator
import warnings
warnings.filterwarnings('ignore')
#h2o.connect()
#h2o.no_progress()
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\eacun\anaconda38\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\eacun\AppData\Local\Temp\tmp2nh5bf0q
  JVM stdout: C:\Users\eacun\AppData\Local\Temp\tmp2nh5bf0q\h2o_eacun_started_from_python.out
  JVM stderr: C:\Users\eacun\AppData\Local\Temp\tmp2nh5bf0q\h2o_eacun_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Halifax
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_eacun_a4k4os
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,241.8 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### I- Applying Logistic Regression to   Vehicle dataset  using  h20

In [2]:
vehicle = h2o.import_file("https://academic.uprm.edu/eacuna/vehicle.csv")
predictors=["COMPACTNESS","CIRCULARITY","DISTANCE_CIRCULARITY","RADIUS_RATIO","PR.AXIS_ASPECT_RATIO","MAX.LENGTH_ASPECT_RATIO","SCATTER_RATIO","ELONGATEDNESS","PR.AXIS_RECTANGULARITY","MAX.LENGTH_RECTANGULARITY","SCALED_VARIANCE_MAJOR","SCALED_VARIANCE_MINOR","SCALED_RADIUS_OF_GYRATION","SKEWNESS_ABOUT_MAJOR","SKEWNESS_ABOUT_MINOR","KURTOSIS_ABOUT_MAJOR","KURTOSIS_ABOUT_MINOR","HOLLOWS_RATIO"]
vehicle['Class']=vehicle['Class'].asfactor()
response_col="Class"
# Create test/train split
#train, test = vehicle.split_frame(ratios=[0.75], seed=1)
glm_model = H2OGeneralizedLinearEstimator(family= "multinomial", lambda_ = 0)
glm_model.train(predictors, response_col, training_frame= vehicle)
glm_model
glm_model.confusion_matrix(vehicle)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,bus,opel,saab,van,Error,Rate
0,214.0,1.0,2.0,1.0,0.018349,4 / 218
1,1.0,142.0,67.0,2.0,0.330189,70 / 212
2,4.0,57.0,154.0,2.0,0.290323,63 / 217
3,1.0,2.0,0.0,196.0,0.015075,3 / 199
4,220.0,202.0,223.0,201.0,0.165485,140 / 846


In [3]:
#Hallando la tasa de prediccion
y_pred=glm_model.predict(vehicle)
print((y_pred['predict']==vehicle['Class']).sum()/len(vehicle))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
0.83451536643026


### II Logistic regression for vehicle using scikit learn

In [4]:
df1=pd.read_csv("http://academic.uprm.edu/eacuna/vehicle.csv")
#Convirtiendo en matriz la tabla de predictoras y la columna de clases
y=df1['Class']
X=df1.iloc[:,0:18]
#Haciendo la regresion logistica y calculando su precision
model = LogisticRegression(solver="lbfgs",max_iter=1000)
model.fit(X,y)
model.score(X,y)

0.8286052009456265

In [5]:
predictions = model.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

         bus       0.96      0.97      0.97       218
        opel       0.71      0.67      0.69       212
        saab       0.68      0.70      0.69       217
         van       0.97      0.98      0.97       199

    accuracy                           0.83       846
   macro avg       0.83      0.83      0.83       846
weighted avg       0.83      0.83      0.83       846



### III. Logistic Regression for Landsat using scikit-learn

In [15]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]
X=data.iloc[:,0:36]
#Haciendo la regresion logistica y calculando su precision
model = LogisticRegression(solver="lbfgs", max_iter=5000)
model = model.fit(X, y)
model.score(X,y)

0.8780157835400225

In [16]:
predictions = model.predict(X)
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           1       0.97      0.98      0.98      1072
           2       0.96      0.95      0.95       479
           3       0.88      0.96      0.92       961
           4       0.61      0.37      0.46       415
           5       0.84      0.82      0.83       470
           6       0.83      0.90      0.86      1038

    accuracy                           0.88      4435
   macro avg       0.85      0.83      0.83      4435
weighted avg       0.87      0.88      0.87      4435



In [17]:
#Estimating the error using the holdout mothod
y1=y.to_numpy()
X1=X.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=0)
X_train, y_train

X_test, y_test

logland= LogisticRegression(solver="lbfgs", max_iter=5000)
logland = logland.fit(X_train, y_train)
logland.score(X_test,y_test)

0.8625093914350113

### IV Logistic Regression for  Landsat using h20

In [18]:
landsat = h2o.import_file("https://academic.uprm.edu/eacuna/landsat.txt")
predictors=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13',
            'C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','C27',
           'C28','C29', 'C30','C31','C32','C33','C34','C35',"C36"]
landsat['C37']=landsat['C37'].asfactor()
response_col="C37"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
glm_model = H2OGeneralizedLinearEstimator(family= "multinomial", lambda_ = 0)
glm_model.train(predictors, response_col, training_frame= landsat)
glm_model
glm_model.confusion_matrix(landsat)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,1,2,3,4,5,6,Error,Rate
0,1049.0,0.0,14.0,2.0,7.0,0.0,0.021455,"23 / 1,072"
1,0.0,462.0,0.0,0.0,17.0,0.0,0.035491,17 / 479
2,5.0,0.0,905.0,49.0,0.0,2.0,0.058273,56 / 961
3,6.0,2.0,90.0,172.0,8.0,137.0,0.585542,243 / 415
4,15.0,11.0,0.0,4.0,392.0,48.0,0.165957,78 / 470
5,0.0,0.0,13.0,69.0,30.0,926.0,0.107900,"112 / 1,038"
6,1075.0,475.0,1022.0,296.0,454.0,1113.0,0.119278,"529 / 4,435"


In [20]:
#Hallando la tasa de prediccion
y_pred=glm_model.predict(landsat)
print((y_pred['predict']==landsat['C37']).sum()/len(landsat))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
0.8807215332581736
